# Composability


In [25]:
import datetime
import time
import functools
import itertools
from dataclasses import dataclass
from enum import Enum, auto
from typing import Callable


## What is composability

In [2]:
# Consider the types used for the definition of a soup from Chapter 9.
# This is an example of data type composition:

class ImperialMeasure(Enum):
    TEASPOON = auto()
    TABLESPOON = auto()
    CUP = auto()

class Broth(Enum):
    VEGETABLE = auto()
    CHICKEN = auto()
    BEEF = auto()
    FISH = auto()

@dataclass(frozen=True)
class Ingredient:
    """
    Ingredients added into the broth
    """
    name: str
    amount: float = 1
    units: ImperialMeasure = ImperialMeasure.CUP

@dataclass
class Recipe:
    aromatics: set[Ingredient]
    broth: Broth
    vegetables: set[Ingredient]
    meats: set[Ingredient]
    starches: set[Ingredient]
    garnishes: set[Ingredient]
    time_to_cook: datetime.timedelta


## Policies vs mechanisms

In [3]:
# When mechanisms are decoupled from policies, we can write declaratively, 
# where we make declarations about what to do, as follows:

def make_potato_leek_and_bacon_soup():
    bacon = bacon_preparer.make_bacon(slices=2)
    potatoes = veg_cheese_preparer.cube_potatoes(grams=300)
    leeks = veg_cheese_preparer.slice(ingredient=Vegetable.LEEKS, grams=250)
    chopped_bacon = chop(bacon)

    # The following methods are provided by soup preparer
    soup_preparer.add_chicken_stock()
    add(potatoes)
    add(leeks)
    cook_for(minutes=30)
    blend()
    garnish(chopped_bacon)
    garnish(Garnish.BLACK_PEPPER)


## Composing on a smalles scale

### Composing Functions

In [4]:
# Consider a function that executes another function twice:

def do_twice(func: Callable, *args, **kwargs):
    result = func(*args, **kwargs)
    result = func(*args, **kwargs)


In [26]:
# We can rewrite the 'do_twice' function into a decorator, as follows:

def do_twice(func: Callable) -> Callable:
    """
    This is a function that calls the wrapped function 2 times
    """
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        for _ in range(2):
            func(*args, **kwargs)
    
    return _wrapper

@do_twice
def say_hello(name):
    print(f"Hello, {name}!!")

say_hello('William')


Hello, William!!
Hello, William!!


In [11]:
# Now consider the following functions that may throw exceptions:

class Dish:
    pass

def on_dish_ordered(dish: Dish):
    dish_db[dish].count += 1

def save_inventory_counts(inventory):
    for ingredient in inventory:
        inventory_db[ingredient.name] = ingredient.count

def log_time_per_dish(dish: Dish, number_of_seconds: int):
    dish_db[dish].time_spent.append(number_of_seconds)


In [17]:
# Before continuing, install and import the 'backoff' and 'requests' packages

"""
!pip install backoff
!pip install requests
"""
import backoff
import requests


In [18]:
# With backoff, we can specify how functions should be retried 
# if they throw an exception, as follows:

class OperationException(Exception):
    pass

class Kitchen:
    def __init__(self) -> None:
        pass

    @backoff.on_exception(backoff.expo, OperationException, max_tries=5)
    def on_dish_ordered(self, dish: Dish):
        self.dish_db[dish].count += 1
    
    @backoff.on_exception(backoff.expo, OperationException, max_tries=3)
    @backoff.on_exception(backoff.expo, requests.exceptions.HTTPError, max_time=60)
    def save_inventory_counts(self, inventory):
        for ingredient in inventory:
            self.inventory_db[ingredient.name] = ingredient.count
    
    @backoff.on_exception(backoff.expo, OperationException, max_time=30)
    def log_time_per_dish(self, dish: Dish, number_of_seconds: int):
        self.dish_db[dish].time_spent.append(number_of_seconds)


In [20]:
# Without the last decorators, the function will be far harder to understand, as follows:

class Kitchen2:
    def __init__(self) -> None:
        pass

    def save_inventory_counts(self, inventory):
        retry = True
        retry_counter = 0
        time_to_sleep = 1
        while retry:
            try:
                for ingredient in inventory:
                    self.inventory_db[ingredient.name] = ingredient.count
            except OperationException:
                retry_counter += 1
                if retry_counter == 5:
                    retry = False
            except requests.exception.HTTPError:
                time.sleep(time_to_sleep)
                time_to_sleep *= 2
                if time_to_sleep > 60:
                    retry = False


### Composing algorithms

Consider the following meal recommendation algorithm:
```
* Look at all daily specials
* Sort based on number of matching surplus ingredients
* Select the meals with the highest number of surplus ingredients
* Sort by proximity to last meal ordered (proximity is defined by number of ingredients that match)
* Take only results that are above 75% proximity
* Return up to top 3 results
```


In [23]:
# We can express the last algorithm in terms of for loops, as follows:

class Meal:
    pass

def get_proximity(x, y):
    return 

def recommend_meal(
    last_meal: Meal,
    specials: list[Meal],
    surplus: list[Ingredient]
) -> list[Meal]:
    highest_proximity = 0
    for special in specials:
        proximity = get_proximity(special, surplus)
        if proximity > highest_proximity:
            highest_proximity = proximity
    
    grouped_by_surplus_matching = []
    for special in specials:
        if get_proximity(special, surplus) == highest_proximity:
            grouped_by_surplus_matching.append(special)
    
    filtered_meals = []
    for meal in grouped_by_surplus_matching:
        if get_proximity(meal, last_meal) > 0.75:
            filtered_meals.append(meal)
    
    sorted_meals = sorted(
        filtered_meals,
        key=lambda meal: get_proximity(meal, last_meal),
        reverse=True
    )
    return sorted_meals[:3]

Consider another meal recommendation algorithm:
```
* Look at all meals available
* Sort based on proximity to last meal
* Select the meals with the highest proximity
* Sort the meals by number of surplus ingredients
* Take only results that are a special or have more than 3 surplus ingredients
* Return up to top 5 result
```

The last 2 algorithms can be generalized as follows:

```
* Look at <a list of meals>
* Sort based on <initial sorting criteria>
* Select the meals with the <grouping criteria>
* Sort the meals by <secondary sorting criteria>
* Take top results that match <selection criteria>
* Return up to top <number> result
```


In [37]:
# With the help of the itertools module, we can translate the generalized
# algorithm into code as follows:

class RecommendationPolicy:
    pass

def recommend_meal(policy: RecommendationPolicy) -> list[Meal]:
    meals = policy.meals
    sorted_meals = sorted(
        meals, key=policy.initial_sorting_criteria,
        reverse=True
    )
    grouped_meals = itertools.groupby(
        sorted_meals, 
        key=policy.grouping_criteria
    )
    _, top_grouped = next(grouped_meals)
    secondary_sorted = sorted(
        top_grouped, 
        key=policy.secondary_sorting_criteria,
        reverse=True
    )
    candidates = itertools.takewhile(policy.selection_criteria, secondary_sorted)
    return list(candidates)[:policy.desired_number_of_recommendations]

# Then, we can use the algorithm as follows:

"""
recommend_meal(
    RecommendationPolicy(
        meals=get_specials(),
        initial_sorting_criteria='get_proximity_to_surplus_ingredients',
        grouping_criteria=get_proximity_to_surplus_ingredients,
        secondary_sorting_criteria=get_proximity_to_last_meal,
        selection_criteria=proximity_greater_than_75_percent,
        desired_number_of_recommendations=3
    )
)
"""


"\nrecommend_meal(\n    RecommendationPolicy(\n        meals=get_specials(),\n        initial_sorting_criteria='get_proximity_to_surplus_ingredients',\n        grouping_criteria=get_proximity_to_surplus_ingredients,\n        secondary_sorting_criteria=get_proximity_to_last_meal,\n        selection_criteria=proximity_greater_than_75_percent,\n        desired_number_of_recommendations=3\n    )\n)\n"